In [3]:
import pyspark
from pyspark.sql import types, functions as F,SparkSession

In [4]:
spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/04/24 16:16:50 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.37.32 instead (on interface eth0)
23/04/24 16:16:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/24 16:16:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## `Spark Udf`

In [2]:
def cubed(num):
    return num*num*num

In [5]:
# registering the udf
spark.udf.register("cubed", cubed, types.LongType())

<function __main__.cubed(num)>

In [8]:
spark.range(1,9).createOrReplaceTempView('cube_view')

In [12]:
spark.sql('select id, cubed(id) as cubed_id from cube_view').show()

+---+--------+
| id|cubed_id|
+---+--------+
|  1|       1|
|  2|       8|
|  3|      27|
|  4|      64|
|  5|     125|
|  6|     216|
|  7|     343|
|  8|     512|
+---+--------+



### `Pandas udf`

In [15]:
from pyspark.sql.functions import pandas_udf

cubed_pd_udf = pandas_udf(cubed, types.LongType())

In [17]:
df = spark.range(1,50)
df.select("id", cubed_pd_udf(F.col("id"))).show()

+---+---------+
| id|cubed(id)|
+---+---------+
|  1|        1|
|  2|        8|
|  3|       27|
|  4|       64|
|  5|      125|
|  6|      216|
|  7|      343|
|  8|      512|
|  9|      729|
| 10|     1000|
| 11|     1331|
| 12|     1728|
| 13|     2197|
| 14|     2744|
| 15|     3375|
| 16|     4096|
| 17|     4913|
| 18|     5832|
| 19|     6859|
| 20|     8000|
+---+---------+
only showing top 20 rows



## `Note`

use spark sbin start-thriftserver, stop-thriftserver , to connect external tools like tableau, dbeaver ,etc to spark tables

## `High order functions`

In [45]:
df = spark.read.text('/home/snehil/Desktop/data-engineering-resources/4_spark/spark_book/data/')

In [54]:
df = df.select('value',F.explode(F.split(df.value, ',').alias('exploded')))

#### transform

* The PySpark sql.functions.transform() is used to apply the transformation on a column of type Array. This function applies the specified transformation on every element of the array and returns an object of ArrayType.

In [74]:
df = spark.read.text('/home/snehil/Desktop/data-engineering-resources/4_spark/spark_book/data/')
df = df.select('value',F.split(df.value, ',').alias('splitted'))
df.show()

+-----------------+--------------------+
|            value|            splitted|
+-----------------+--------------------+
|State,Color,Count|[State, Color, Co...|
|        TX,Red,20|       [TX, Red, 20]|
|       NV,Blue,66|      [NV, Blue, 66]|
|       CO,Blue,79|      [CO, Blue, 79]|
|       OR,Blue,71|      [OR, Blue, 71]|
|     WA,Yellow,93|    [WA, Yellow, 93]|
|       WY,Blue,16|      [WY, Blue, 16]|
|     CA,Yellow,53|    [CA, Yellow, 53]|
|      WA,Green,60|     [WA, Green, 60]|
|      OR,Green,71|     [OR, Green, 71]|
|      TX,Green,68|     [TX, Green, 68]|
|      NV,Green,59|     [NV, Green, 59]|
|      AZ,Brown,95|     [AZ, Brown, 95]|
|     WA,Yellow,20|    [WA, Yellow, 20]|
|       AZ,Blue,75|      [AZ, Blue, 75]|
|      OR,Brown,72|     [OR, Brown, 72]|
|        NV,Red,98|       [NV, Red, 98]|
|     WY,Orange,45|    [WY, Orange, 45]|
|       CO,Blue,52|      [CO, Blue, 52]|
|      TX,Brown,94|     [TX, Brown, 94]|
+-----------------+--------------------+
only showing top

In [78]:
df.select(df.splitted,F.transform('splitted', lambda x:F.upper(x)).alias('splitted_UPPER')).show(truncate=False)

+---------------------+---------------------+
|splitted             |splitted_UPPER       |
+---------------------+---------------------+
|[State, Color, Count]|[STATE, COLOR, COUNT]|
|[TX, Red, 20]        |[TX, RED, 20]        |
|[NV, Blue, 66]       |[NV, BLUE, 66]       |
|[CO, Blue, 79]       |[CO, BLUE, 79]       |
|[OR, Blue, 71]       |[OR, BLUE, 71]       |
|[WA, Yellow, 93]     |[WA, YELLOW, 93]     |
|[WY, Blue, 16]       |[WY, BLUE, 16]       |
|[CA, Yellow, 53]     |[CA, YELLOW, 53]     |
|[WA, Green, 60]      |[WA, GREEN, 60]      |
|[OR, Green, 71]      |[OR, GREEN, 71]      |
|[TX, Green, 68]      |[TX, GREEN, 68]      |
|[NV, Green, 59]      |[NV, GREEN, 59]      |
|[AZ, Brown, 95]      |[AZ, BROWN, 95]      |
|[WA, Yellow, 20]     |[WA, YELLOW, 20]     |
|[AZ, Blue, 75]       |[AZ, BLUE, 75]       |
|[OR, Brown, 72]      |[OR, BROWN, 72]      |
|[NV, Red, 98]        |[NV, RED, 98]        |
|[WY, Orange, 45]     |[WY, ORANGE, 45]     |
|[CO, Blue, 52]       |[CO, BLUE, 

#### filter

* The filter() function produces an array consisting of only the elements of the input
array for which the Boolean function is true

In [86]:
df.select(df.splitted, F.filter('splitted', lambda x:F.length(x)>2).alias('color')).show()

+--------------------+--------------------+
|            splitted|               color|
+--------------------+--------------------+
|[State, Color, Co...|[State, Color, Co...|
|       [TX, Red, 20]|               [Red]|
|      [NV, Blue, 66]|              [Blue]|
|      [CO, Blue, 79]|              [Blue]|
|      [OR, Blue, 71]|              [Blue]|
|    [WA, Yellow, 93]|            [Yellow]|
|      [WY, Blue, 16]|              [Blue]|
|    [CA, Yellow, 53]|            [Yellow]|
|     [WA, Green, 60]|             [Green]|
|     [OR, Green, 71]|             [Green]|
|     [TX, Green, 68]|             [Green]|
|     [NV, Green, 59]|             [Green]|
|     [AZ, Brown, 95]|             [Brown]|
|    [WA, Yellow, 20]|            [Yellow]|
|      [AZ, Blue, 75]|              [Blue]|
|     [OR, Brown, 72]|             [Brown]|
|       [NV, Red, 98]|               [Red]|
|    [WY, Orange, 45]|            [Orange]|
|      [CO, Blue, 52]|              [Blue]|
|     [TX, Brown, 94]|          

#### exists

* The exists() function returns true if the Boolean function holds for any element in
the input array

In [89]:
df.select(df.splitted,F.exists('splitted',lambda x:x=='TX').alias('state is TX')).show()

+--------------------+-----------+
|            splitted|state is TX|
+--------------------+-----------+
|[State, Color, Co...|      false|
|       [TX, Red, 20]|       true|
|      [NV, Blue, 66]|      false|
|      [CO, Blue, 79]|      false|
|      [OR, Blue, 71]|      false|
|    [WA, Yellow, 93]|      false|
|      [WY, Blue, 16]|      false|
|    [CA, Yellow, 53]|      false|
|     [WA, Green, 60]|      false|
|     [OR, Green, 71]|      false|
|     [TX, Green, 68]|       true|
|     [NV, Green, 59]|      false|
|     [AZ, Brown, 95]|      false|
|    [WA, Yellow, 20]|      false|
|      [AZ, Blue, 75]|      false|
|     [OR, Brown, 72]|      false|
|       [NV, Red, 98]|      false|
|    [WY, Orange, 45]|      false|
|      [CO, Blue, 52]|      false|
|     [TX, Brown, 94]|       true|
+--------------------+-----------+
only showing top 20 rows



## Also available functions include join , union ,windowing ,etc